In [249]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from collections import Counter
from itertools import chain

In [250]:
# nltk.download()

In [251]:
violations_section = pd.read_csv("../data/violations_section.csv")
violations_TCA = pd.read_csv("../data/violations_TCA.csv")
violations_MCL = pd.read_csv("../data/violations_MCL.csv")
violations_description_id = pd.read_csv("../data/violations_description_id.csv")
violations_remainder = pd.read_csv("../data/violations_remainder.csv")

In [252]:
violations_list = [violations_section, violations_TCA, violations_MCL, violations_description_id, violations_remainder]

In [253]:
for violation in violations_list:
    assert 'Unnamed: 0' in violation.columns

In [254]:
for violation in violations_list:
    violation.set_index('Unnamed: 0', inplace=True)
    violation.index.rename('Original_Index', inplace=True)

In [255]:
violations_description_id.head()

,id,type,desc,text,remedial
Original_Index,,,,,
15,(B). It is unlawful for any person to dispose ...,NaN,N,NaN,1.0
438,he or she shall serve notice of the risk and t...,NaN,NaN,NaN,NaN
439,tree at the expense of the property owner,NaN,N,NaN,2.0
707,"human occupation or use, upon failure or refus...",NaN,NaN,NaN,NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,NaN,2.0


In [256]:
violations_description_id['text'] = violations_description_id['id'].values

In [257]:
violations_description_id.head()

,id,type,desc,text,remedial
Original_Index,,,,,
15,(B). It is unlawful for any person to dispose ...,NaN,N,(B). It is unlawful for any person to dispose ...,1.0
438,he or she shall serve notice of the risk and t...,NaN,NaN,he or she shall serve notice of the risk and t...,NaN
439,tree at the expense of the property owner,NaN,N,tree at the expense of the property owner,2.0
707,"human occupation or use, upon failure or refus...",NaN,NaN,"human occupation or use, upon failure or refus...",NaN
708,"remove or demolish, such dwelling or structure...",NaN,N,"remove or demolish, such dwelling or structure...",2.0


In [258]:
for violation in violations_list:
    violation.drop('id', axis='columns', inplace=True)
    
    violation['desc'].replace(np.nan, '', regex=True, inplace=True)
    violation['text'].replace(np.nan, '', regex=True, inplace=True)

In [259]:
violations_section.head()

,type,desc,text,remedial
Original_Index,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE


In [260]:
violations_TCA.head()

,type,desc,text,remedial
Original_Index,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN


In [261]:
violations_MCL.head()

,type,desc,text,remedial
Original_Index,,,,
481,CAGA_DEX,DOG EXCREMENT,MCL Chapter 8.04.180 – Removal of Excrement. ...,NaN
482,CAGB_HAZ_W,HAZARDOUS WASTE,"MCL Chapter 10.20.150 - Hazardous, pathogenic ...",NaN
483,CAGC_GCL,GARBAGE CONTAINER LIDS,MCL Chapter 10.20.160 – Container Requirements...,NaN
484,CAGD_ID,ILLEGAL DUMPING,MCL Chapter 10.20.320 – Illegal Dumping. It i...,NaN
485,CAGE_LPR,LITTER ON PRIVATE PROPERTY,MCL Chapter 10.24.070 - Litter on private prop...,NaN


In [262]:
violations_description_id.head()

,type,desc,text,remedial
Original_Index,,,,
15,NaN,N,(B). It is unlawful for any person to dispose ...,1.0
438,NaN,,he or she shall serve notice of the risk and t...,NaN
439,NaN,N,tree at the expense of the property owner,2.0
707,NaN,,"human occupation or use, upon failure or refus...",NaN
708,NaN,N,"remove or demolish, such dwelling or structure...",2.0


In [263]:
violations_remainder.head()

,type,desc,text,remedial
Original_Index,,,,
126,MVENVEND,Mobile Vending,Ordinance BL2006-1283 - Mobile Vendor: The ped...,CAAZ_MVEN_VENDING
196,TRASHCANS,Trash Cans,Ordinance 89-826 - Trash Cans: Refuse containe...,CAAH_TRASH_CANS
209,SWVIOLPTP,Violations - Post Construction Treatment,See SW Code,NaN
210,SWVIOLWQ,Violations for Water Quality Impacts,See SW Code,NaN
211,SWENVCRT,Environmental Court Proceedings,See SW Code,NaN


In [264]:
master_violations = pd.concat(violations_list)

In [265]:
master_violations

,type,desc,text,remedial
Original_Index,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE
5,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU
6,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG
7,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT
8,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP


In [266]:
def tokenize_column(column_values):
    return word_tokenize(column_values)
    
def tokenize_row(index, row):
    desc = row['desc']
    text = row['text']
    
    desc_tokenized, text_tokenized = [], []
    
    if desc:
        desc_tokenized = tokenize_column(desc)
        
    if text:
        text_tokenized = tokenize_column(text)

    return (index, desc_tokenized, text_tokenized)

In [267]:
tokenized_data = list()

for index, row in master_violations.iterrows():
    tokenized_row_data = tokenize_row(index, row)
    tokenized_data.append(tokenized_row_data)

In [268]:
tokenized_df = pd.DataFrame(tokenized_data, columns=['Original_Index', 'tokenized_desc', 'tokenized_text'])
tokenized_df.set_index('Original_Index', inplace=True)

tokenized_df.head()

,tokenized_desc,tokenized_text
Original_Index,,
0,[Banners],"[Section, 17.32.060, &, 17.32.070, -, Banners,..."
1,"[Proper, Maintenance, Req, -, Owner]","[Section, 16.16.030, (, B, ), -, Proper, maint..."
2,"[Proper, Maintenance, Req]","[Section, 16.16.030, (, A, ), -, Proper, maint..."
3,"[Building, Permit, Required]","[Section, 16.28.010, -, Building, Permit, Requ..."
4,"[Scope, of, Building, Code]","[Section, 16.16.020, (, B, ), -, The, provisio..."


In [269]:
master_violations = master_violations.join(tokenized_df)
master_violations.head()

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
0,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,[Banners],"[Section, 17.32.060, &, 17.32.070, -, Banners,..."
1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,"[Proper, Maintenance, Req, -, Owner]","[Section, 16.16.030, (, B, ), -, Proper, maint..."
2,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,"[Proper, Maintenance, Req]","[Section, 16.16.030, (, A, ), -, Proper, maint..."
3,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,"[Building, Permit, Required]","[Section, 16.28.010, -, Building, Permit, Requ..."
4,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,"[Scope, of, Building, Code]","[Section, 16.16.020, (, B, ), -, The, provisio..."


In [270]:
#subset that contains rows with the regular expression "Section #." in text column
master_section = master_violations[master_violations.text.str.match(r"(S|s)ection\s(\d+)\.", na=False)]

#new df where rows containing "Section #" are removed
master_nosection = master_violations.drop(master_section.index)

#subset that contains rows with variations of TCA in text column
master_TCA = master_nosection[master_nosection.text.str.match("(T.C.A|Tennessee Code Annotated)", na=False)]

#new df where rows containing "T.C.A" or Tennessee Code Annotated" are removed
master_noTCA = master_nosection.drop(master_TCA.index)

#subset that contains variations of MCL in text column 
#Be aware MCL and MCL Chapter appear to be different types of codes
master_MCL = master_noTCA[master_noTCA.text.str.match("(M.C.L|MCL Chapter)", na=False)]

#new df where rows containing "M.C.L" or "MCL Chapter" are removed
master_remainder = master_noTCA.drop(master_MCL.index)
#master_remainder

In [271]:
#export csv files containing subsets for section, TCA, MCL, descriptions in id column, and remaining entries.
master_section.to_csv("../data/master_section.csv")
master_TCA.to_csv("../data/master_TCA.csv")
master_MCL.to_csv("../data/master_MCL.csv")
master_remainder.to_csv("../data/master_remainder.csv")

In [272]:
master_TCA

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG,"[Landlord, Registration, Required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN,"[LANDLORD, REGISTRATION, REQUIRED]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN,"[Not, Manufacturer, and, Wholesale, Retail]","[T.C.A, 57-5-101, (, a, ), (, 2, ), :, Except,..."
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN,"[Not, Wholesale, and, Manufacturer, Retail]","[T.C.A, 57-5-101, (, a, ), (, 3, ), :, Except,..."
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN,"[Operating, without, county, city, permit]","[T.C.A, 57-5-103, (, a, ), (, 1, ), :, It, is,..."
810,BT575103A10,Permits to citizen legal resident,"T.C.A 57-5-103(a)(10): After July 1, 2015, a c...",NaN,"[Permits, to, citizen, legal, resident]","[T.C.A, 57-5-103, (, a, ), (, 10, ), :, After,..."
811,BT575103A2,Permit to business owner,T.C.A 57-5-103(a)(2): Permits shall be issued ...,NaN,"[Permit, to, business, owner]","[T.C.A, 57-5-103, (, a, ), (, 2, ), :, Permits..."
812,BT575103A3A,Cannot transfer ownership,T.C.A 57-5-103(a)(3)(A): A permit shall be val...,NaN,"[Can, not, transfer, ownership]","[T.C.A, 57-5-103, (, a, ), (, 3, ), (, A, ), :..."
813,BT575103A3B,Permit for single location only,T.C.A 57-5-103(a)(3)(B): A permit shall be val...,NaN,"[Permit, for, single, location, only]","[T.C.A, 57-5-103, (, a, ), (, 3, ), (, B, ), :..."


In [273]:
master_TCA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 106 to 882
Data columns (total 6 columns):
type              74 non-null object
desc              74 non-null object
text              74 non-null object
remedial          1 non-null object
tokenized_desc    74 non-null object
tokenized_text    74 non-null object
dtypes: object(6)
memory usage: 4.0+ KB


In [274]:
#converting to lower case and remonving stop words
for i in range(0,len(master_TCA)):
    vodes = master_TCA.iloc[i, 4]
    newlist = []
    for item in vodes:
        newlist.append(item.lower())
    master_TCA.iloc[i, 4] = newlist
        

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [275]:
master_TCA.head()

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG,"[landlord, registration, required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN,"[landlord, registration, required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN,"[not, manufacturer, and, wholesale, retail]","[T.C.A, 57-5-101, (, a, ), (, 2, ), :, Except,..."
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN,"[not, wholesale, and, manufacturer, retail]","[T.C.A, 57-5-101, (, a, ), (, 3, ), :, Except,..."
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN,"[operating, without, county, city, permit]","[T.C.A, 57-5-103, (, a, ), (, 1, ), :, It, is,..."


In [276]:
TCA = master_TCA
TCA.head()

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG,"[landlord, registration, required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN,"[landlord, registration, required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
807,BT575101A2,Not Manufacturer and Wholesale Retail,T.C.A 57-5-101(a)(2): Except as otherwise prov...,NaN,"[not, manufacturer, and, wholesale, retail]","[T.C.A, 57-5-101, (, a, ), (, 2, ), :, Except,..."
808,BT575101A3,Not Wholesale and Manufacturer Retail,T.C.A 57-5-101(a)(3): Except as otherwise prov...,NaN,"[not, wholesale, and, manufacturer, retail]","[T.C.A, 57-5-101, (, a, ), (, 3, ), :, Except,..."
809,BT575103A1,Operating without county city permit,T.C.A 57-5-103(a)(1): It is unlawful to operat...,NaN,"[operating, without, county, city, permit]","[T.C.A, 57-5-103, (, a, ), (, 1, ), :, It, is,..."


In [277]:
#stop = list(stopwords.words('english'))



#for item in range(0,len(TCA)):
    #vodesstop = TCA.iloc[item, 4]
    #newliststop = []
    #if stop in vodesstop:
        #newliststop.replace(stop, '')
    #TCA.iloc[item, 4] = newliststop


In [278]:
#converting the tokenized description column into. a list, which results into a column as a list with eac row in that 
#column as a list
TCA_list = TCA["tokenized_desc"].tolist()
TCA_list

[['landlord', 'registration', 'required'],
 ['landlord', 'registration', 'required'],
 ['not', 'manufacturer', 'and', 'wholesale', 'retail'],
 ['not', 'wholesale', 'and', 'manufacturer', 'retail'],
 ['operating', 'without', 'county', 'city', 'permit'],
 ['permits', 'to', 'citizen', 'legal', 'resident'],
 ['permit', 'to', 'business', 'owner'],
 ['can', 'not', 'transfer', 'ownership'],
 ['permit', 'for', 'single', 'location', 'only'],
 ['business', 'name', 'same', 'as', 'permit', 'name'],
 ['two', 'businesses', 'one', 'location', 'one', 'permit'],
 ['on', 'sale', 'off', 'sale', 'one', 'permit'],
 ['return', 'within', '15', 'days', 'of', 'termination'],
 ['no', 'valid', 'cert', 'to', 'sell', ',', 'dist', ',', 'manufacture'],
 ['resale', 'cert', 'not', 'displayed', 'distributed'],
 ['sales', 'near', 'schools', ',', 'churches'],
 ['no', 'sales', 'to', 'minors'],
 ['law', 'violations', 'by', 'owners', 'in', 'past', '10', 'yrs'],
 ['employee', 'law', 'violations', 'in', 'past', '10', 'yrs'],


In [280]:
#stopwords = set(stopwords.words('english'))
stoplist=[]
for item in TCA_list:
    wr=[]
    for words in item:
        if words not in stop:
            wr.append(word)
        stoplist.append(wr)
        #print(wr)
print(TCA_list)           

[['landlord', 'registration', 'required'], ['landlord', 'registration', 'required'], ['not', 'manufacturer', 'and', 'wholesale', 'retail'], ['not', 'wholesale', 'and', 'manufacturer', 'retail'], ['operating', 'without', 'county', 'city', 'permit'], ['permits', 'to', 'citizen', 'legal', 'resident'], ['permit', 'to', 'business', 'owner'], ['can', 'not', 'transfer', 'ownership'], ['permit', 'for', 'single', 'location', 'only'], ['business', 'name', 'same', 'as', 'permit', 'name'], ['two', 'businesses', 'one', 'location', 'one', 'permit'], ['on', 'sale', 'off', 'sale', 'one', 'permit'], ['return', 'within', '15', 'days', 'of', 'termination'], ['no', 'valid', 'cert', 'to', 'sell', ',', 'dist', ',', 'manufacture'], ['resale', 'cert', 'not', 'displayed', 'distributed'], ['sales', 'near', 'schools', ',', 'churches'], ['no', 'sales', 'to', 'minors'], ['law', 'violations', 'by', 'owners', 'in', 'past', '10', 'yrs'], ['employee', 'law', 'violations', 'in', 'past', '10', 'yrs'], ['onsale', 'permit

In [281]:
# as the above list each row is a sublist so the dict.counter will not be able to count it thua we need to change each 
#sublist to an elemnt in the big list
TCA_flat_list = [item for sublist in TCA_list for item in sublist]
TCA_flat_list

['landlord',
 'registration',
 'required',
 'landlord',
 'registration',
 'required',
 'not',
 'manufacturer',
 'and',
 'wholesale',
 'retail',
 'not',
 'wholesale',
 'and',
 'manufacturer',
 'retail',
 'operating',
 'without',
 'county',
 'city',
 'permit',
 'permits',
 'to',
 'citizen',
 'legal',
 'resident',
 'permit',
 'to',
 'business',
 'owner',
 'can',
 'not',
 'transfer',
 'ownership',
 'permit',
 'for',
 'single',
 'location',
 'only',
 'business',
 'name',
 'same',
 'as',
 'permit',
 'name',
 'two',
 'businesses',
 'one',
 'location',
 'one',
 'permit',
 'on',
 'sale',
 'off',
 'sale',
 'one',
 'permit',
 'return',
 'within',
 '15',
 'days',
 'of',
 'termination',
 'no',
 'valid',
 'cert',
 'to',
 'sell',
 ',',
 'dist',
 ',',
 'manufacture',
 'resale',
 'cert',
 'not',
 'displayed',
 'distributed',
 'sales',
 'near',
 'schools',
 ',',
 'churches',
 'no',
 'sales',
 'to',
 'minors',
 'law',
 'violations',
 'by',
 'owners',
 'in',
 'past',
 '10',
 'yrs',
 'employee',
 'law',
 '

In [188]:
#to count the frequecy of the key words in the dict we will use the counter function
TCA_list_freq = dict(Counter(TCA_flat_list))
TCA_list_freq

{',': 3,
 '1': 1,
 '10': 2,
 '12': 1,
 '12-6': 1,
 '15': 1,
 '2': 2,
 '21': 2,
 'action': 1,
 'address': 1,
 'advertising': 1,
 'agreement': 2,
 'and': 3,
 'application': 1,
 'approval': 1,
 'as': 2,
 'at': 1,
 'bankruptcy': 1,
 'beer': 12,
 'between': 1,
 'bond': 3,
 'brand': 1,
 'business': 2,
 'businesses': 1,
 'by': 1,
 'can': 2,
 'cancel': 1,
 'cancelled': 1,
 'cert': 2,
 'churches': 1,
 'citizen': 1,
 'city': 1,
 'clerk': 1,
 'coerce': 2,
 'compensation': 1,
 'conditions': 1,
 'consent': 3,
 'consumption': 1,
 'contest': 1,
 'contests': 1,
 'contraband': 1,
 'conviction': 1,
 'county': 1,
 'credit': 1,
 'days': 1,
 'delay': 2,
 'delivery': 2,
 'displayed': 1,
 'dist': 1,
 'distance': 1,
 'distributed': 1,
 'documentation': 1,
 'employee': 1,
 'exceed': 1,
 'exhibition': 1,
 'failure': 3,
 'false': 1,
 'fees': 1,
 'felony': 1,
 'financial': 1,
 'flavored': 2,
 'for': 4,
 'from': 1,
 'gallon': 1,
 'get': 1,
 'homemade': 9,
 'hotel': 2,
 'id': 1,
 'identified': 1,
 'in': 3,
 'indica

In [189]:
TCA_kw_s = pd.Series(TCA_list_freq, name="freq")
TCA_kw_s

,                3
1                1
10               2
12               1
12-6             1
15               1
2                2
21               2
action           1
address          1
advertising      1
agreement        2
and              3
application      1
approval         1
as               2
at               1
bankruptcy       1
beer            12
between          1
bond             3
brand            1
business         2
businesses       1
by               1
can              2
cancel           1
cancelled        1
cert             2
churches         1
                ..
sign             2
single           1
site             1
state            1
statement        1
storage          1
stored           1
suppler          2
supplier        11
suspended        1
termination      2
threaten         1
to              10
transfer         9
two              1
unauthorized     1
under            2
units            1
unreasonable     1
valid            1
vendor           4
violations  

In [190]:
#convert series to the dataframe
TCA_kw = pd.DataFrame(data = TCA_kw_s, columns = ["freq"])
TCA_kw = TCA_kw.reset_index()
TCA_kw.columns = ["kw", "freq"]
TCA_kw

,kw,freq
0,",",3
1,1,1
2,10,2
3,12,1
4,12-6,1
5,15,1
6,2,2
7,21,2
8,action,1
9,address,1


In [284]:
TCA_kw_sorted = TCA_kw.sort_values(by = "freq", ascending = False)
TCA_kw_sorted

,kw,freq
18,beer,12
154,supplier,11
158,to,10
65,homemade,9
159,transfer,9
113,permit,8
169,wholesaler,7
96,not,6
124,required,5
140,sales,5


In [300]:
searchfor = ['beer', 'homemade', 'supplier']
TCA_drop_beer = TCA.drop(TCA[TCA["desc"].str.contains('|'.join(searchfor), case = False)].index)
TCA_drop_beer.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 106 to 882
Data columns (total 6 columns):
type              51 non-null object
desc              51 non-null object
text              51 non-null object
remedial          1 non-null object
tokenized_desc    51 non-null object
tokenized_text    51 non-null object
dtypes: object(6)
memory usage: 2.8+ KB


In [302]:
searchfor = ['homemade']
TCA_drop_homemade = TCA.drop(TCA[TCA["desc"].str.contains('|'.join(searchfor), case = False)].index)
TCA_drop_homemade.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66 entries, 106 to 882
Data columns (total 6 columns):
type              66 non-null object
desc              66 non-null object
text              66 non-null object
remedial          1 non-null object
tokenized_desc    66 non-null object
tokenized_text    66 non-null object
dtypes: object(6)
memory usage: 3.6+ KB


In [350]:
searchfor = [
    'beer', 'supplier', 'vendor', 'homemade', 'permit', 'sale', 'sales', 'seller', 'wholesale', 'wholesaler', 'minors', 'minor',
    "transfer", "manufacturer", "suspension", "hotel", "cert", "certification",
    'documentation']
TCA_drop = TCA.drop(TCA[TCA["desc"].str.contains('|'.join(searchfor), case = False)].index)

#s[s.str.contains('|'.join(searchfor))]

#MCL_noBB = MCL_MCL.drop(MCL_MCL[MCL_MCL["desc"].str.contains("BB ")].index) 

In [351]:
print(TCA_drop)

                         type                                     desc  \
Original_Index                                                           
106                   LANDREG           Landlord Registration Required   
376             CAAR_LAND_REG           LANDLORD REGISTRATION REQUIRED   
817                BT575103A6     Return within 15 days of termination   
822                BT575105B3  Law violations by owners in past 10 yrs   
823                BT575105B4   Employee law violations in past 10 yrs   
825                 BT575105D           False statement on application   
828                 BT575110A                            Bond required   
829                 BT575110B                      Bond rider required   
830                BT575110E2                          Bond revocation   
842                 BT575206A            Invoice and memoranda keeping   
849                 BT575304A                      Only 1 outdoor sign   
857                BT5755032E     Supp

In [352]:
# out of 74 total classifed in master_TCA renamed as TCA after putting all things as lower case
#53 are classified by the key words chossed 
#two entries will fall with supplier are spelled wrong


TCA_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 106 to 882
Data columns (total 6 columns):
type              18 non-null object
desc              18 non-null object
text              18 non-null object
remedial          1 non-null object
tokenized_desc    18 non-null object
tokenized_text    18 non-null object
dtypes: object(6)
memory usage: 1008.0+ bytes


In [353]:
searchfor1 = ['bond', 'terminate', 'pay', 'violations', 'possession', 'consent', 'distance', 'termination']
TCA_drop_rest = TCA_drop.drop(TCA_drop[TCA_drop["desc"].str.contains('|'.join(searchfor1), case = False)].index)
TCA_drop_rest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 106 to 858
Data columns (total 6 columns):
type              7 non-null object
desc              7 non-null object
text              7 non-null object
remedial          1 non-null object
tokenized_desc    7 non-null object
tokenized_text    7 non-null object
dtypes: object(6)
memory usage: 392.0+ bytes


In [354]:
TCA_drop_rest

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
106,LANDREG,Landlord Registration Required,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,CAAR_LAND_REG,"[landlord, registration, required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
376,CAAR_LAND_REG,LANDLORD REGISTRATION REQUIRED,Tennessee Code Annotated (TCA) 66-28-101 - Fai...,NaN,"[landlord, registration, required]","[Tennessee, Code, Annotated, (, TCA, ), 66-28-..."
825,BT575105D,False statement on application,T.C.A 57-5-105(d): Any applicant making a fals...,NaN,"[false, statement, on, application]","[T.C.A, 57-5-105, (, d, ), :, Any, applicant, ..."
842,BT575206A,Invoice and memoranda keeping,"T.C.A 57-5-206(a): Every person, firm, corpora...",NaN,"[invoice, and, memoranda, keeping]","[T.C.A, 57-5-206, (, a, ), :, Every, person, ,..."
849,BT575304A,Only 1 outdoor sign,"T.C.A 57-5-304(a): No outdoor sign, advertisem...",NaN,"[only, 1, outdoor, sign]","[T.C.A, 57-5-304, (, a, ), :, No, outdoor, sig..."
857,BT5755032E,Suppler require conditions or limits,T.C.A 57-5-503(2)(E): Require a wholesaler to ...,NaN,"[suppler, require, conditions, or, limits]","[T.C.A, 57-5-503, (, 2, ), (, E, ), :, Require..."
858,BT5755032F,Suppler require financial information,T.C.A 57-5-503(2)(F): Require a wholesaler to ...,NaN,"[suppler, require, financial, information]","[T.C.A, 57-5-503, (, 2, ), (, F, ), :, Require..."


In [355]:
searchfor2 = ['landlord', 'sign']
TCA_drop_rest2 = TCA_drop_rest.drop(TCA_drop_rest[TCA_drop_rest["desc"].str.contains('|'.join(searchfor2), case = False)].index)
TCA_drop_rest2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 825 to 858
Data columns (total 6 columns):
type              4 non-null object
desc              4 non-null object
text              4 non-null object
remedial          0 non-null object
tokenized_desc    4 non-null object
tokenized_text    4 non-null object
dtypes: object(6)
memory usage: 224.0+ bytes


In [356]:
TCA_drop_rest2
#there are 2 entries perhaps duplicate for landlord
#there are 2 misspelled 'suppler'
#

,type,desc,text,remedial,tokenized_desc,tokenized_text
Original_Index,,,,,,
825,BT575105D,False statement on application,T.C.A 57-5-105(d): Any applicant making a fals...,NaN,"[false, statement, on, application]","[T.C.A, 57-5-105, (, d, ), :, Any, applicant, ..."
842,BT575206A,Invoice and memoranda keeping,"T.C.A 57-5-206(a): Every person, firm, corpora...",NaN,"[invoice, and, memoranda, keeping]","[T.C.A, 57-5-206, (, a, ), :, Every, person, ,..."
857,BT5755032E,Suppler require conditions or limits,T.C.A 57-5-503(2)(E): Require a wholesaler to ...,NaN,"[suppler, require, conditions, or, limits]","[T.C.A, 57-5-503, (, 2, ), (, E, ), :, Require..."
858,BT5755032F,Suppler require financial information,T.C.A 57-5-503(2)(F): Require a wholesaler to ...,NaN,"[suppler, require, financial, information]","[T.C.A, 57-5-503, (, 2, ), (, F, ), :, Require..."


In [358]:
#TCA: 74 entries look for these key words 
    #['beer', 'supplier', 'vendor', 'homemade', 'permit', 'sale', 'sales', 'seller', 'wholesale', 'wholesaler', 'minors', 'minor',
    #"transfer","manufacturer", "suspension", "hotel", "cert", "certification",
    #'documentation']
#TCA_drop: 18 entries look for ['bond', 'terminate', 'pay', 'violations', 'possession', 'consent', 
    #'distance', 'termination']
#TCA_drop_rest: 7 entries look for ['landlord', 'sign']
#TCA_drop_rest2: 4 entries left 2 misspelled suppler, 1 false statement, 1 invoice memoranda
